In [30]:
from collections import deque
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
# from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.preprocessing import StandardScaler

import ta
# from ta import add_all_ta_features
# from ta.momentum import RSIIndicator
# from ta.volatility import BollingerBands
# from ta.trend import MACD

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from backtesting import Backtest, Strategy
# from backtesting.lib import crossover


# 1 - Load data

In [31]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    # Xử lý datetime
    # df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    # df = df.sort_values('Datetime').drop(['Date', 'Time'], axis=1)
    df['Datetime'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Datetime').drop(['Date'], axis=1)
    df = df.set_index('Datetime', drop=True)
    return df

data = load_data('data/xauusd/1m/dynamic_labeled_test.csv')

In [32]:
data = data.iloc[:20000]
data

,Open,High,Low,Close,Volume,Label
Datetime,,,,,,
2022-01-03 00:05:00,1830.63,1830.64,1829.65,1829.73,45,HOLD
2022-01-03 00:06:00,1829.75,1830.16,1829.65,1830.02,56,HOLD
2022-01-03 00:07:00,1829.90,1829.95,1829.52,1829.79,47,HOLD
2022-01-03 00:08:00,1829.87,1831.13,1829.80,1831.13,45,SELL
2022-01-03 00:09:00,1831.13,1831.82,1830.86,1831.65,58,SELL
...,...,...,...,...,...,...
2022-01-21 16:11:00,1837.92,1837.92,1836.97,1837.27,184,SELL
2022-01-21 16:12:00,1837.27,1837.43,1835.29,1835.84,361,BUY
2022-01-21 16:13:00,1835.85,1835.93,1835.32,1835.80,267,BUY


In [33]:
def add_technical_indicators(df):
    """Add technical indicators."""
    # Momentum
    df['RSI'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    df['Momentum'] = ta.momentum.ROCIndicator(df['Close']).roc()
    df['CMO'] = ta.momentum.kama(df['Close'])
    df['Williams_%R'] = ta.momentum.WilliamsRIndicator(df['High'], df['Low'], df['Close']).williams_r()
    # Volatility
    df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()
    bb = ta.volatility.BollingerBands(df['Close'])
    df['BB_Mid'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['BB_Bandwidth'] = bb.bollinger_wband()
    keltner = ta.volatility.KeltnerChannel(df['High'], df['Low'], df['Close'])
    df['KC_High'] = keltner.keltner_channel_hband()
    df['KC_Low'] = keltner.keltner_channel_lband()
    donchian = ta.volatility.DonchianChannel(df['High'], df['Low'], df['Close'])
    df['DC_High'] = donchian.donchian_channel_hband()
    df['DC_Low'] = donchian.donchian_channel_lband()
    # Trend
    df['SMA_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['EMA_20'] = ta.trend.EMAIndicator(df['Close'], window=20).ema_indicator()
    df['DPO'] = ta.trend.DPOIndicator(df['Close']).dpo()
    macd = ta.trend.MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Hist'] = macd.macd_diff()
    df['Mass_Index'] = ta.trend.mass_index(df['High'], df['Low'])
    # Volume
    df['AD'] = ta.volume.AccDistIndexIndicator(df['High'], df['Low'], df['Close'], df['Volume']).acc_dist_index()
    df['CMF'] = ta.volume.ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume']).chaikin_money_flow()
    df['Force_Index'] = ta.volume.ForceIndexIndicator(df['Close'], df['Volume']).force_index()
    df['MFI'] = ta.volume.MFIIndicator(df['High'], df['Low'], df['Close'], df['Volume']).money_flow_index()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()

    return df.reset_index(drop=True)

def add_basic_features(df):
    """Final preprocessing: pct_change, time features."""
    # Calculate percentage change for OHLC
    cols_to_pct = ['Open', 'High', 'Low', 'Close']
    existing_cols = [col for col in cols_to_pct if col in df.columns]
    if existing_cols:
        df[existing_cols] = df[existing_cols].pct_change().fillna(0) * 100
        df['Cum_Return'] = df['Close'].rolling(window=20).sum()
        df['Cum_Turnover'] = df['Volume'].rolling(window=20).sum()

    # Add time features
    if 'Datetime' in df.columns:
        df['Hour'] = df['Datetime'].dt.hour / 23.0
        df['Day_Of_Week'] = df['Datetime'].dt.dayofweek / 6.0
        df['Minute_Of_Day'] = (df['Datetime'].dt.hour * 60 + df['Datetime'].dt.minute) / 1439.0

    return df.reset_index(drop=True)

def normalize_by_blocks(data, block_size):
    if isinstance(data, pd.DataFrame):
        columns = data.columns
        index = data.index
        data_np = data.values.astype('float32')
    else:
        data_np = data.astype('float32')
        columns = None
        index = None

    result = np.zeros_like(data_np)
    num_blocks = 0

    for start_idx in range(0, len(data_np), block_size):
        end_idx = min(start_idx + block_size, len(data_np))
        block = data_np[start_idx:end_idx]

        if block.shape[0] > 0:
            scaler = StandardScaler()
            if block.shape[0] == 1:
                normalized_block = block - np.mean(block, axis=0)
            else:
                std_devs = np.std(block, axis=0)
                if np.any(std_devs == 0):
                    normalized_block = np.zeros_like(block)
                    valid_cols = std_devs != 0
                    if np.any(valid_cols):
                        scaler.fit(block[:, valid_cols])
                        normalized_block[:, valid_cols] = scaler.transform(block[:, valid_cols])
                    zero_std_cols = std_devs == 0
                    if np.any(zero_std_cols):
                        normalized_block[:, zero_std_cols] = block[:, zero_std_cols] - np.mean(block[:, zero_std_cols], axis=0)
                else:
                    normalized_block = scaler.fit_transform(block)

            if np.isnan(normalized_block).any() or np.isinf(normalized_block).any():
                normalized_block = np.nan_to_num(normalized_block, nan=0.0, posinf=0.0, neginf=0.0)

            result[start_idx:end_idx] = normalized_block
            num_blocks += 1

    if columns is not None and index is not None:
        return pd.DataFrame(result, columns=columns, index=index)
    else:
        return result


# 2 - Load model

In [34]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, 32, kernel_size=1, padding='same')
        self.branch3 = nn.Conv1d(in_channels, 32, kernel_size=3, padding='same')
        self.branch5 = nn.Conv1d(in_channels, 32, kernel_size=5, padding='same')
        self.branch_pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, 32, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch3(x), self.branch5(x), self.branch_pool(x)], dim=1)

class Time2Vec(nn.Module):
    def __init__(self, time_dim, kernel_dim=32):
        super().__init__()
        self.linear = nn.Linear(time_dim, 1)
        self.periodic = nn.Linear(time_dim, kernel_dim - 1)

    def forward(self, x):
        linear = self.linear(x)
        periodic = self.periodic(x)
        return torch.cat([linear, periodic], dim=-1)

class CrossAttentionFusion(nn.Module):
    def __init__(self, cnn_dim, transformer_dim):
        super().__init__()
        self.query = nn.Linear(cnn_dim, transformer_dim)
        self.key = nn.Linear(transformer_dim, transformer_dim)
        self.value = nn.Linear(transformer_dim, transformer_dim)
        
    def forward(self, cnn_features, transformer_features):
        Q = self.query(cnn_features).unsqueeze(1)  # [batch, 1, transformer_dim]
        K = self.key(transformer_features)         # [batch, seq_len, transformer_dim]
        V = self.value(transformer_features)       # [batch, seq_len, transformer_dim]
        
        attn_scores = (Q @ K.transpose(-2, -1)) / (K.size(-1) ** 0.5)  # [batch, 1, seq_len]
        attn_weights = torch.softmax(attn_scores, dim=-1)
        
        return torch.bmm(attn_weights, V).squeeze(1)  # [batch, transformer_dim]

class HighwayNetwork(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Sigmoid()
        )
        
    def forward(self, fused, transformer):
        g = self.gate(fused)
        return g * fused + (1 - g) * transformer

class EnhancedHybridModel(nn.Module):
    def __init__(self, num_features, time_dim, num_classes=3, d_model=512, nhead=16, dim_feedforward=1024, num_layers=6):
        super().__init__()
        # 1. InceptionTime Branch
        self.inception = nn.Sequential(
            InceptionModule(num_features),
            nn.ReLU(),
            nn.MaxPool1d(2),
            InceptionModule(128),
            nn.ReLU()
        )
        
        # 2. Transformer Branch
        self.time2vec = Time2Vec(time_dim, d_model//2)
        self.transformer_proj = nn.Linear(num_features, d_model//2)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 3. Fusion
        self.cross_attention = CrossAttentionFusion(128, d_model)
        self.highway = HighwayNetwork(d_model)
        
        # 4. Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, time_feature):
        # 1. Inception Path
        cnn_features = self.inception(x.permute(0, 2, 1))  # [batch, channels, seq_len//2]
        cnn_features = cnn_features.mean(dim=-1)          # [batch, channels=128]
        
        # 2. Transformer Path
        time_embed = self.time2vec(time_feature)  # [batch, seq_len, d_model // 2]
        x_proj = self.transformer_proj(x)  # [batch, seq_len, d_model // 2]
        combined = torch.cat([time_embed, x_proj], dim=-1) # [batch, seq_len, d_model]
        transformer_features = self.transformer(combined)  # [batch, seq_len, d_model]
        
        # 3. Fusion
        fused = self.cross_attention(cnn_features, transformer_features)  # [batch, d_model]
        output = self.highway(fused, transformer_features.mean(dim=1))   # [batch, d_model]
        
        return self.classifier(output)

In [35]:
# ...existing code...

class InceptionTransformerStrategy(Strategy):
    STOP_LOSS = 0.01  # 1% stop-loss
    TAKE_PROFIT = 0.02  # 2% take-profit
    SEQ_LEN = 128
    CONFIDENCE_THRESHOLD = 0.7
    # Chọn các cột đặc trưng phù hợp với mô hình của bạn
    FEATURE_COLS = [
        'Open', 'High', 'Low', 'Close', 'Volume', 
        'RSI', 'Momentum', 'CMO', 'Williams_%R', 'ATR',
        'BB_Mid', 'BB_Upper', 'BB_Lower', 'BB_Bandwidth',
        'KC_High', 'KC_Low', 'DC_High', 'DC_Low',
        'SMA_20', 'EMA_20', 'DPO', 'MACD', 'MACD_Hist', 'Mass_Index',
        'AD', 'CMF', 'Force_Index', 'MFI', 'OBV', 'Cum_Return', 'Cum_Turnover',
        'Hour', 'Day_Of_Week', 'Minute_Of_Day'
    ]
    # DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    DEVICE = torch.device('cpu')
    PATH = 'save/models/best_model.pth'

    def init(self):
        self.n_features = len(self.FEATURE_COLS)
        self.time_dim = 3  # Hour, Day_Of_Week, Minute_Of_Day
        self.model = EnhancedHybridModel(
            num_features=self.n_features, time_dim=1, num_classes=3, 
            d_model=128, nhead=8, dim_feedforward=256, num_layers=3
        )
        self.model.load_state_dict(torch.load(self.PATH, map_location=self.DEVICE))
        self.model.eval()
        self.model.to(self.DEVICE)
        self.buffer = deque(maxlen=self.SEQ_LEN)
        self.temp_df = pd.DataFrame(columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    def next(self):
        new_data = {
            'Datetime': self.data.index[-1],
            'Open': self.data.Open[-1],
            'High': self.data.High[-1],
            'Low': self.data.Low[-1],
            'Close': self.data.Close[-1],
            'Volume': self.data.Volume[-1]
        }
        self.temp_df = pd.concat([self.temp_df, pd.DataFrame([new_data])], ignore_index=True)

        if len(self.temp_df) > 13:
            temp_df = self.temp_df.copy()
            temp_df = add_technical_indicators(temp_df)
            temp_df = add_basic_features(temp_df)

            if temp_df[self.FEATURE_COLS].iloc[-1].notna().all():
                self.buffer.append(temp_df[self.FEATURE_COLS].iloc[-1].values.astype(np.float32))

            if len(self.buffer) == self.SEQ_LEN:
                sequence = np.array(self.buffer, dtype=np.float32)
                sequence = normalize_by_blocks(sequence, block_size=self.SEQ_LEN)

                time_idx = [self.FEATURE_COLS.index('Minute_Of_Day')]
                time_feature = sequence[:, time_idx]
                time_feature = normalize_by_blocks(time_feature, block_size=self.SEQ_LEN)

                feature_tensor = torch.tensor(sequence, dtype=torch.float32, device=self.DEVICE).unsqueeze(0)
                time_tensor = torch.tensor(time_feature, dtype=torch.float32, device=self.DEVICE).unsqueeze(0)

                with torch.no_grad():
                    logits = self.model(feature_tensor, time_tensor)
                    probs = torch.softmax(logits, dim=-1).squeeze().cpu().numpy()

                signal = int(np.argmax(probs))
                sorted_probs = np.sort(probs)
                confidence_margin = sorted_probs[-1] - sorted_probs[-2]

                current_price = self.data.Close[-1]
                MARGIN_THRESHOLD = 0.4

                if confidence_margin > MARGIN_THRESHOLD:
                    # print('OK', signal)
                    if signal == 0 and not self.position:  # BUY
                        self.buy(
                            size=0.1,
                            sl=current_price * (1 - self.STOP_LOSS),
                            tp=current_price * (1 + self.TAKE_PROFIT)
                        )
                        # print('BUY')
                    elif signal == 1 and not self.position:  # SELL
                        self.sell(
                            size=0.1,  
                            sl=current_price * (1 + self.STOP_LOSS),
                            tp=current_price * (1 - self.TAKE_PROFIT)
                        )
                        # print('SELL')
                if signal == 2 and self.position:  # CLOSE nếu tự tin muốn giữ tiền mặt
                    self.position.close()
                    # print('HOLD')
                # print('------------')


# ...existing code...

In [36]:
# import backtesting
# import multiprocessing

bt = Backtest(
    data,
    InceptionTransformerStrategy,
    cash=10000,
    commission=.0002,
    exclusive_orders=True
)

stats = bt.run()
print(stats)
bt.plot()


Backtest.run:   0%|          | 0/19999 [00:00<?, ?bar/s]

Start                     2022-01-03 00:05:00
End                       2022-01-21 16:15:00
Duration                     18 days 16:10:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                 0.39733
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                      

GridPlot(id='p1333', ...)